# In this notebook, we compile all programs used to generate power from tree trunk temperature differences. To run the code, hit "Run" above, and make sure to run all cells above the target cell.

## The structure of the notebook is :

  1. import all libraries and packages
  2. import all data files in csv format
  3. define hyperparameters
  4. define parameters
  5. generate boundary condition data
  6. main time stepping code
  7. compute generated current
  8. visualization

Import packages; numpy and scipy are for simulations; pandas for reading data; matplotlib and mpl_tookits.mplot3d for visualization. All experiments are run in Python3.6.9.

In [1]:
import numpy as np
import pandas
from scipy import sparse
from scipy.optimize import broyden1
from functools import partial
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D # matplotlib version 3.1.2
%matplotlib inline

Define hyperparameters- these are non-physical parameters whose values are used to control the simulation process via the Crank-Nicolson method. In particular, we define the number of grid points, and the number of time steps. There is no built in CFL check, as Crank-Nicolson is unconditionally stable. 

**Need to look at the definition of n_t**

In [2]:
def make_hyper_parameters(n_x, n_t, plot_at, diff_at, radius, height):
    """
    Argument:
    n_x -- number of grid points, radially; interior points index from 1 to n_x - 1; bdry points are 0 and n_x.
    n_t -- number of time steps
    plot_at -- grid point at which a temperature curve is drawn
    diff_at -- list of two grid points where the temp difference is computed
    radius -- radius of tree, m
    height -- height of tree, m
    sigma -- Stefan–Boltzmann constant, 5.67 × 10 − 8 W ⋅ m − 2 ⋅ K − 4
    
    Returns:
    hyper_parameters -- python dictionary containing hyper parameters:
                    n_x -- number of grid points, radially
                    n_t -- number of time steps
                    time -- time sample from 0 to n_t
                    plot_at -- example grid point 
                    diff_at -- temperature difference
                    radius -- radius of tree, m
                    height -- height of tree, m
                    dr -- step size 
    """
    time = np.linspace(0, 24, n_t, endpoint = False)
    dt = time[1] - time[0]
    
    # Boundary conditions at trunk[0] and trunk[n_x]; Temperature[j] at trunk[j]; in matrix, only index from 1 to n_x-1 (labeled m)
    trunk = np.linspace(0, radius, n_x+1)   
    dr = radius / n_x
    
    hyper_parameters = {"n_x": int(n_x),
                  "n_t": int(n_t),
                  "time": time,
                  "trunk": trunk,
                  "plot_at": plot_at,
                  "diff_at": diff_at,
                  "radius": radius,
                  "height": height,
                  "dr": dr,
                  "dt": dt}
    
    return hyper_parameters  

In [3]:
# this cell is help to understand/explain the trunk vector. Delete when done
r = 0.18
n_x = 160
trunk = np.linspace(0, r, n_x + 1)
itemindex = np.where(trunk==trunk[-2])
print(trunk[0],trunk[1],trunk[-1],trunk[n_x],trunk.shape,itemindex,trunk[itemindex],trunk[itemindex] + trunk[1])

0.0 0.001125 0.18 0.18 (161,) (array([159]),) [0.178875] [0.18]


In [4]:
# test hyper parameters
hyper_parameters = make_hyper_parameters(160, 1000, 53, [1, -3], 0.18, 10)

print("n_x = " + str(hyper_parameters["n_x"]))
print("n_t = " + str(hyper_parameters["n_t"]))
print("time.shape = " + str(hyper_parameters["time"].shape))
print("trunk.shape = " + str(hyper_parameters["trunk"].shape))
print("radius = " + str(hyper_parameters["radius"]))
print("height = " + str(hyper_parameters["height"]))
print("plot_at = " + str(hyper_parameters["plot_at"]))
print("diff_at = " + str(hyper_parameters["diff_at"]))
print("dr = " + str(hyper_parameters["dr"]))
print("dt = " + str(hyper_parameters["dt"]))

n_x = 160
n_t = 1000
time.shape = (1000,)
trunk.shape = (161,)
radius = 0.18
height = 10
plot_at = 53
diff_at = [1, -3]
dr = 0.001125
dt = 0.024


Import data from TNTweather.csv; we use data within 24 hours from Feb 16, 2021. Data includes date, time, temperature (ambient), windspeed, and solar radiation; measured starting from 0 am to 23:59pm. With piece wise linear interpolation. This piece is hard coded due to the format of data.

In [5]:
def make_weather_input(n_t):
    
    colnames = ['date', 'time', 'temperature', 'windspeed', 'solar']

    dataWeather = pandas.read_csv('TNTweather.csv', names = colnames)

    #date16 = dataWeather.date[4:27].tolist()
    #time16 = dataWeather.time[4:27].tolist()

    temp16 = dataWeather.temperature[4:27].tolist() # row 6 ro 29
    temp16np = np.asarray([float(t) for t in temp16])

    windspeed16 = dataWeather.windspeed[4:27].tolist()
    windspeed16np = np.asarray([float(v) for v in windspeed16])

    solar16 = dataWeather.solar[4:27].tolist()
    solar16np = np.asarray([float(v) for v in solar16])

    air_temp = np.interp(np.linspace(0,24,n_t), np.linspace(0,24,temp16np.size),temp16np)
    windspeed = np.interp(np.linspace(0,24,n_t), np.linspace(0,24,windspeed16np.size),windspeed16np)
    solar = np.interp(np.linspace(0,24,n_t), np.linspace(0,24,solar16np.size),solar16np)
    
    weather_input = {"air_temp": air_temp,
                  "windspeed": windspeed,
                  "solar": solar}
    return weather_input

In [6]:
# to run in code: weather_input = weather_input(hyper_parameters["n_t"])
weather_input = make_weather_input(2)
print("air_temp = " + str(weather_input["air_temp"]))
print("windspeed = " + str(weather_input["windspeed"]))
print("solar = " + str(weather_input["solar"]))

air_temp = [300.15 300.15]
windspeed = [3.601108 3.086664]
solar = [0. 0.]


Import tree temperature data from Protasio, including core and bark temperature; use data measured within 24 hours from Feb 16, 2021.

In [7]:
def make_temp_input(n_t):
    colnames = ['date1', 'time1', 'barkTemp', 'coreTemp']
    
    dataTemp = pandas.read_csv('TNTtemp.csv', names=colnames)
    
    #datet16 = dataTemp.date1[367:781].tolist()
    #timet16 = dataTemp.time1[367:781].tolist()

    coreTemp16 = dataTemp.coreTemp[367:781].tolist() # row 6 ro 29
    coreTemp16np = np.asarray([float(t) for t in coreTemp16])

    barkTemp16 = dataTemp.barkTemp[367:781].tolist()
    barkTemp16np = np.asarray([float(v) for v in barkTemp16])
    
    bark_temp = np.interp(np.linspace(0,24,n_t), np.linspace(0,24,barkTemp16np.size),barkTemp16np)
    core_temp = np.interp(np.linspace(0,24,n_t), np.linspace(0,24,coreTemp16np.size),coreTemp16np)
    diff_temp = np.average(bark_temp - core_temp)
    
    temp_input = {"bark_temp": bark_temp,
                  "core_temp": core_temp,
                  "diff_temp": diff_temp}
    
    return temp_input    

In [8]:
temp_input = make_temp_input(10)
print("bark_temp = " + str(temp_input["bark_temp"]))
print("core_temp = " + str(temp_input["core_temp"]))
print("diff_temp = " + str(temp_input["diff_temp"]))

bark_temp = [300.21 300.84 302.9  303.84 303.65 302.4  301.71 301.27 300.92 300.59]
core_temp = [300.64 300.43 300.45 301.01 301.45 301.79 301.89 301.7  301.47 301.14]
diff_temp = 0.6359999999999844


Define parameters used in time stepping- related to the properties of the trees and weather: thermal conductivity (W/(mK), $\rho$ the density (kg/m^3), specific heat (J/(kgK)); $\rho$ mutiplies specific heat gives the heat capacity; finally
albedo which impacts how much solar radiation is absorbed (taken to be 0.3 from Potter and Andresen 2002).

We choose a constant thermal conductivity, as numerical experiments show little difference.

In [9]:
def make_step_parameters(Kt, rho, specific_heat, albedo):
    """
    Argument:
    Kt -- thermal conductivity of the tree, unit W/(mK)
    rho -- density of tree trunk, unit kg/m^3
    specific_heat -- specific_heat, unit J/(kgK)
    albedo -- albedo constant; taken to be 0.3 from Potter Andresen pg.3
    
    Returns:
    step_parameters -- python dictionary containing hyper parameters:
                    thermal_conductivity -- thermal conductivity vector
                    heat_capacity -- rho * specific_heat
                    albedo -- albedo constant
    """
    thermal_conductivity = Kt  
    
    # may delete if can broadcast
    heat_capacity = rho * specific_heat
    
    step_parameters = {"Kt": Kt,
                  "heat_capacity": heat_capacity,
                  "albedo": albedo}
    
    return step_parameters  

In [10]:
# test time stepping parameters
step_parameters = make_step_parameters(0.12, 510, 1380, 0.3)

print("Kt = " + str(step_parameters["Kt"]))
print("heat_capacity = " + str(step_parameters["heat_capacity"]))
print("albedo = " + str(step_parameters["albedo"]))

Kt = 0.12
heat_capacity = 703800
albedo = 0.3


In the one dimensional model, we see that temperature distribution depends on time $t$ and distance from the center $r$. 

\begin{equation}
\rho c\frac{\partial T}{\partial t}=\frac{1}{r}\frac{\partial}{\partial r}\bigg(kr\frac{\partial T}{\partial r}\bigg)+\text{ source terms}
=\frac{\partial k}{\partial r}\frac{\partial T}{\partial r}+\frac{k}{r}\frac{\partial T}{\partial r}+k\frac{\partial^2T}{\partial r^2}+\text{ source terms}.\label{heat1d}
\end{equation}

Let $T^n_{i} \approx T(r_i,t_n)$ denote the numerical solution at the point $(r_i,t_n)$, $\Delta t$ the time step, and $\Delta r$ grid size. In this formulation, we have taken the thermal conductivity to be constant within the tree trunk. We denote the total source term to be $g_s(r,t)$, and use the source value $g_s(t)\frac{1}{\Delta r}$, because the sources are all surface energy fluxes, while the conduction terms apply to the model element volume. **experiment with/out division of Delta r**. 


With second order central difference in space 
\begin{equation}\frac{\partial T}{\partial r}\approx\frac{T_{i+1}-T_{i-1}}{2\Delta r}\end{equation}
\begin{equation}\frac{\partial^2 T}{\partial r^2}\approx\frac{T_{i-1}-2T_{i}+T_{i+1}}{(\Delta r)^2}\end{equation}
and first order forward in time
\begin{equation}\frac{\partial T}{\partial t}\approx\frac{T^1-T^0}{\Delta t},\end{equation}
we obtain the Crank-Nicolson method by averaging the above spatial differences in time (no source):
\begin{align}
&(T^{n+1}_i - T^n_i)/\Delta t = 1/(4ar_i\Delta r)(T^n_{i+1}-T^n_{i-1} + T^{n+1}_{i+1}-T^{n+1}_{i-1} )\nonumber\\
&+1/(2a(\Delta r)^2)(T^n_{i-1} - 2T^n_i + T^n_{i+1} + T^{n+1}_{i-1} - 2T^{n+1}_i + T^{n+1}_{i+1}),\ (1)\label{1dCNscheme}
\end{align}
where $a = \frac{\rho c}{k}$. With the source value, we obtain

\begin{align}
&(T^{n+1}_i - T^n_i)/\Delta t = 1/(4ar_i\Delta r)(T^n_{i+1}-T^n_{i-1} + T^{n+1}_{i+1}-T^{n+1}_{i-1} )\nonumber\\
&+1/(2a(\Delta r)^2)(T^n_{i-1} - 2T^n_i + T^n_{i+1} + T^{n+1}_{i-1} - 2T^{n+1}_i + T^{n+1}_{i+1})+\frac{1}{2\rho c\Delta r}(g_s(r_i,t) + g_s(r_i,t+\Delta t)),\ (2)\label{1dCNschemeSource}
\end{align}

Separating terms from two time steps gives the following
\begin{equation}
\alpha_i T^1_{i-1} +(1+\beta)T^1_i-\gamma_iT^1_{i+1}=-\alpha_i T^0_{i-1} +(1-\beta)T^0_i+\gamma_iT^0_{i+1}+\frac{\Delta t}{2\rho c\Delta r}(g_s(r_i,t) + g_s(r_i,t+\Delta t)),\ (3)\end{equation}
where $\beta = \frac{\Delta t}{a(\Delta r)^2}$, $\alpha_i = \frac{\Delta t}{4ar_i\Delta r}-\frac{\Delta t}{2a(\Delta r)^2}$, and $\gamma_i = \frac{\Delta t}{4ar_i\Delta r}+\frac{\Delta t}{2a(\Delta r)^2}$

We implement two types of boundary conditions at the inner boundary:

1. no-flux condition
\begin{equation}
    \frac{\partial T}{\partial r}\bigg|_{r=0}=0,
\end{equation} 

for numerical simplicity (referenced in FireStem2D). Numerically, we use the second order central difference approximation $ \frac{\partial T}{\partial r}\bigg|_{r=0}\approx \frac{T_2-T_0}{2\Delta r}=0,$, and the above equation gives a representation for the ghost cell value $T_0$ in terms of $T_2$. 

2. we enforce experimental data measured at the center of the tree trunk
\begin{equation}
    T(r=0, t)=g_0(t),\ \text{ measured data},
\end{equation} 

For the outer boundary, we implement three types of boundary condition:

1. with experimental data measured at location (Dirichlet condition)
\begin{equation}
    T(r=R, t)=g_1(t),\ \text{ measured data},
\end{equation} 

2. with experimental data measured at location, but approximated by the measured air temperature.
\begin{equation}
    T(r=0, t)=t_{air}(t),\ \text{ approximate air temperature},
\end{equation} 

3. with simulated data generated with the steady state solution (?) **read Slack history**
\begin{equation}
    T(r=0, t)=t_{simulated}(t),\ \text{ simulated data},
\end{equation} 

The first row of the following matrix equation implements the inner boundary condtion, and the last row the outter boundary condition. The source term is also added to the last row. 

The following equations are similar to equation (9.9) in LeVeque's book on Finite Difference methods. Here the matrix is of size $n_x -1\times n_x -1$, with $T[0]$ the core and $T[n_x]$ the bark temperature that we do not solve- these are either read from experimental data, or simulated from weather data and pre obtained parameters. Here the index satisfies $n_x-1=m$. (for example, when $n_x = 160$, $m=159$,we solve for temperature values indexed 1 to 159)

If we apply Neumann condition at the center, and Dirichlet boundary condition at the tree bark:

\begin{equation*}
    \left( \begin{array}{cccc}
1+\beta & -\beta & & 0\\
\alpha_2 & 1+\beta & -\gamma_2 & \\
\ddots& \ddots & \ddots  & \\
& \alpha_{m-1} & 1+\beta & -\gamma_{m-1} \\
0 & & \alpha_m & 1+\beta \end{array} \right)\left(\begin{array}{c}
     T^1_1  \\
     T^1_2 \\
     \vdots\\
     T^1_{m-1}\\
     T^1_m
\end{array}\right)=\left( \begin{array}{cccc}
1-\beta & \beta & & 0\\
-\alpha_2 & 1-\beta & \gamma_2 & \\
\ddots& \ddots & \ddots  & \\
& -\alpha_{m-1} & 1-\beta & \gamma_{m-1} \\
0 & & -\alpha_m & 1-\beta \end{array} \right)\left(\begin{array}{c}
     T^0_1  \\
     T^0_2 \\
     \vdots\\
     T^0_{m-1}\\
     T^0_m
\end{array}\right)+\left(\begin{array}{c}
     0  \\
     0 \\
     \vdots\\
     0\\
     \gamma_m(g_1(t)+g_1(t+\Delta t))+\frac{\Delta t}{2\rho c\Delta r}(g_s(t)+g_s(t+\Delta t))
\end{array}\right).
\end{equation*}


Notice that the first row of the matrix represents the inner boundary condition: we first approximate $\partial T/\partial r \approx (T_2-T_0)/(2\Delta r)=0$, then the relation involving $T_1$ and $T_2$ is derived from equation (2) using $T_0 = T_2$. 

The last row of the matrix represents the outer boundary condition: $T(r = R, t) = g_1(t)$, and equation (2) implies
\begin{equation}
\alpha_m T^1_{m-1}+(1+\beta)T^1_m-\gamma_mg_1(t+\Delta t)=-\alpha_m T^0_{m-1}+(1-\beta)T^0_m+\gamma_mg_1(t)
\end{equation}
when we replace the ghost cell $T(r = R+\Delta r, t) \approx T^0_{m+1}$ by $g_1(t)$, and this becomes 
\begin{equation}
\alpha_m T^1_{m-1}+(1+\beta)T^1_m=-\alpha_m T^0_{m-1}+(1-\beta)T^0_m+\gamma_m(g_1(t) +g_1(t+\Delta t))
\end{equation}

*idea: try integrate Matlab and use ode23/45 for MOL*

Similarly, if we apply Dirichlet boundary condition at both the center and the tree bark,

\begin{equation*}
    \left( \begin{array}{cccc}
1+\beta & -\gamma_1 & & 0\\
\alpha_2 & 1+\beta & -\gamma_2 & \\
\ddots& \ddots & \ddots  & \\
& \alpha_{m-1} & 1+\beta & -\gamma_{m-1} \\
0 & & \alpha_m & 1+\beta \end{array} \right)\left(\begin{array}{c}
     T^1_1  \\
     T^1_2 \\
     \vdots\\
     T^1_{m-1}\\
     T^1_m
\end{array}\right)=\left( \begin{array}{cccc}
1-\beta & \gamma_1 & & 0\\
-\alpha_2 & 1-\beta & \gamma_2 & \\
\ddots& \ddots & \ddots  & \\
& -\alpha_{m-1} & 1-\beta & \gamma_{m-1} \\
0 & & -\alpha_m & 1-\beta \end{array} \right)\left(\begin{array}{c}
     T^0_1  \\
     T^0_2 \\
     \vdots\\
     T^0_{m-1}\\
     T^0_m
\end{array}\right)+\left(\begin{array}{c}
     -\alpha_1(g_0(t)+g_0(t+\Delta t))  \\
     0 \\
     \vdots\\
     0\\
     \gamma_m(g_1(t)+g_1(t+\Delta t))+\frac{\Delta t}{2\rho c\Delta r}(g_s(t)+g_s(t+\Delta t))
\end{array}\right).
\end{equation*}

Define the right hand side vectors for Neumann and Dirichlet conditions. Consider the system $AT^1 = BT^0 + v$, first row of $v$ would be 0 if Neumann at the core; and $-\alpha_1(T^1_0+T^0_0)$ if Dirichlet at core. Since both formulation uses Dirichlet outter boundary condition, and source terms are also at the bark, the last row in both cases is $\gamma_m(T^1_{m+1}+T^0_{m+1})+\frac{\Delta t}{2\rho c}(f_s(R,t)+f_s(R,t + \Delta t))$

In [11]:
# matrix needs to be made so that it is of size n_x - 1 by n_x - 1. m = n_x - 1

def time_stepping(hyper_parameters, step_parameters, bdry_type):
    
    sigma = 5.67e-8
    
    rhoc = step_parameters["heat_capacity"]
    Kt = step_parameters["Kt"]
    albedo = step_parameters["albedo"]
    n_x = hyper_parameters["n_x"]
    n_t = hyper_parameters["n_t"]
    dt = hyper_parameters["dt"]
    dr = hyper_parameters["dr"]
    trunk = hyper_parameters["trunk"]
    a = rhoc / Kt
    beta = dt / (a * dr ** 2)
    coeffSource = dt / (2 * rhoc)
    
    # import bdry condition
    temp_input = make_temp_input(n_t)
    core_temp = temp_input["core_temp"]
    bark_temp = temp_input["bark_temp"]
    
    # import weather condition
    weather_input = make_weather_input(n_t)
    air_temp = weather_input["air_temp"]
    windspeed = weather_input["windspeed"]
    solar = weather_input["solar"]
    
    # use h = 21 as an estimate
    h = 21
    convect = h * (air_temp - bark_temp)
    
    """
    compute source terms and then sum
    solar radiation = (1-albedo) * solar
    blackbody/long radiation = sigma * (Tair^4 - Tsfc^4)
    convection = h * (Tsfc - Tair)
    """
    # convect might change based on experimental data to reflect h * (Tsfc - Tair)

    sourceTerm = (1 - albedo) * solar + sigma * (air_temp**4 - bark_temp**4) + convect
    
    T0 = 300.40 * np.ones(n_x) # initialize temperature with an estimate temperature

    
    alpha = np.linspace(1, n_x - 1, n_x - 1)# index from 0 to n_x - 2; cor. to points at 1 to n_x - 1
    for j in range(alpha.size):
        alpha[j] = dt / (4 * a * trunk[j + 2] * dr) - 0.5 * beta
    
    gamma = np.linspace(1, n_x - 1, n_x - 1)
    for j in range(gamma.size):
        gamma[j] = dt / (4 * a * trunk[j + 1] * dr) + 0.5 * beta
    
    # for the right hand side vector- alpha1 if Dirichlet at center; gammam is used for Dirichlet at the bark
    
    alpha1 = dt / (4 * a * trunk[1] * dr) - 0.5 * beta
    gammam = dt / (4 * a * trunk[-1] * dr) + 0.5 * beta
    
#     vAddm = gammam * () + coeffSource * ()
    # system of Ax_{n+1} = Bx_{n} + v    
    
    if bdry_type == "neumann":
        gamma[0] = beta  # define inner neumann bdry condition 1st row
        
   
    elif bdry_type == "dirichlet": 
        gamma[0] = dt / (4 * a * trunk[1] * dr) + 0.5 * beta
#         vAdd1 = -alpha1 * ()
        
        
    
#     print(bdry_type, gamma[0])
    A = sparse.diags([alpha, np.ones(n_x) + beta, -gamma], [-1, 0, 1], shape=(n_x, n_x)).toarray()
    B = sparse.diags([-alpha, np.ones(n_x) - beta, gamma], [-1, 0, 1], shape=(n_x, n_x)).toarray()
    
    T1 = np.zeros((n_x))
    
    soln = []
    
    ## time stepping: step1 compute B * T0; step2 rhs = B * T0 + v; step2 solve for T1 with T1 = A^-1 * rhs
    for time in range(n_t - 1):
        rhs = B.dot(T0)
        rhs[-1] = rhs[-1] + gammam * (bark_temp[time] + bark_temp[time + 1]) + coeffSource * (sourceTerm[time] + sourceTerm[time + 1])
        if bdry_type == "dirichlet": 
            rhs[0] = rhs[0] - alpha1 * (core_temp[time] + core_temp[time + 1])
        
        T1 = np.linalg.solve(A, rhs) 
        T0 = T1
        
        soln.append(T0) # start saving the solutions from time step 1 
    
    step_in_time = {"A": A,
                     "B": B,
                     "beta": beta,
                     "alpha": alpha,
                     "gamma": gamma,
                     "sourceTerm": sourceTerm,
                     "solution": soln}
    
    return step_in_time  
    

In [16]:
# test for different boundary conditions

step_neumann = time_stepping(hyper_parameters, step_parameters, bdry_type = "neumann")

solutionN = step_neumann["solution"]

solutionArrayN = np.asarray(solutionN)
solutionArrayN.shape

step_dirichlet = time_stepping(hyper_parameters, step_parameters, bdry_type = "dirichlet")

solutionD = step_dirichlet["solution"]

solutionArrayD = np.asarray(solutionD)


# find where the max and min are attained in neumann and dirichlet

# for j in range(999):
    

# counter = 0
# for j in range(999):
#     check if Neumann difference is consistently larger than dirichlet
#     if solutionArrayN[j,:].max() - solutionArrayN[j,:].min() <= solutionArrayD[j,:].max() - solutionArrayD[j,:].min():
#         print("Max difference for Neumann is smaller at step", j)
#         counter += 1
#     print()
# print(counter)

Max difference for Neumann is smaller at step 0

Max difference for Neumann is smaller at step 1

Max difference for Neumann is smaller at step 2

Max difference for Neumann is smaller at step 3

Max difference for Neumann is smaller at step 4

Max difference for Neumann is smaller at step 5

Max difference for Neumann is smaller at step 6

Max difference for Neumann is smaller at step 7

Max difference for Neumann is smaller at step 8

Max difference for Neumann is smaller at step 9

Max difference for Neumann is smaller at step 10

Max difference for Neumann is smaller at step 11

Max difference for Neumann is smaller at step 12

Max difference for Neumann is smaller at step 13

Max difference for Neumann is smaller at step 14

Max difference for Neumann is smaller at step 15

Max difference for Neumann is smaller at step 16

Max difference for Neumann is smaller at step 17

Max difference for Neumann is smaller at step 18

Max difference for Neumann is smaller at step 19

Max differ


Max difference for Neumann is smaller at step 740

Max difference for Neumann is smaller at step 741

Max difference for Neumann is smaller at step 742

Max difference for Neumann is smaller at step 743

Max difference for Neumann is smaller at step 744

Max difference for Neumann is smaller at step 745

Max difference for Neumann is smaller at step 746

Max difference for Neumann is smaller at step 747

Max difference for Neumann is smaller at step 748

Max difference for Neumann is smaller at step 749

Max difference for Neumann is smaller at step 750

Max difference for Neumann is smaller at step 751

Max difference for Neumann is smaller at step 752

Max difference for Neumann is smaller at step 753

Max difference for Neumann is smaller at step 754

Max difference for Neumann is smaller at step 755

Max difference for Neumann is smaller at step 756

Max difference for Neumann is smaller at step 757

Max difference for Neumann is smaller at step 758

Max difference for Neumann is 